## Importing required libraries

In [3]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pickle
import numpy.ma as ma
import lightgbm as lgb
#from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
#from sklearn.utils import shuffle
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import dataprofiler
from sklearn.metrics import multilabel_confusion_matrix

## Reading the extracted the dataset

In [4]:
dataset_feb04=pd.read_csv("10k_Final_dataset.csv")
dataset_feb04=dataset_feb04.drop(["Unnamed: 0"],axis=1)
dataset_feb04=dataset_feb04.drop_duplicates()
dataset_feb04=dataset_feb04.reset_index()
dataset_feb04=dataset_feb04.drop(["index"],axis=1)
dataset_feb04

,patient_id,diagnosis_coded,concept_id,value_numeric,value_coded
0,16359994,1385083,1482681,0.883785,\N
1,16359994,1385083,1520177,74,\N
2,16359994,1385083,1520177,75,\N
3,16359994,1385083,1520177,77,\N
4,16359994,1385083,1520177,69,\N
...,...,...,...,...,...
6442641,19999784,1384995,1520694,32.1,\N
6442642,19999784,1384995,1520696,160,\N
6442643,19999784,1384995,1520694,31.6,\N
6442644,19999784,1384995,1520731,\N,1502522


## Taking only the diagnosis column and patient columns

In [5]:
dataset_feb13=dataset_feb04[["patient_id","diagnosis_coded"]]
dataset_feb13=dataset_feb13.drop_duplicates()
dataset_feb13=dataset_feb13.reset_index()
dataset_feb13=dataset_feb13.drop(["index"],axis=1)
dataset_feb13

,patient_id,diagnosis_coded
0,16359994,1385083
1,16360107,1385109
2,16360107,1385121
3,16360107,1474651
4,16360581,1384886
...,...,...
13016,19998497,1397715
13017,19999379,1397715
13018,19999379,1384870
13019,19999625,1385121


## Trying to see total how many patients and how many diagnoses

In [6]:
print("Number of patients in the dataset",len(dataset_feb13["patient_id"].unique()))
print("Number of diagnoses in the dataset",len(dataset_feb13["diagnosis_coded"].unique()))

Number of patients in the dataset 10000
Number of diagnoses in the dataset 10


## Trying to if there is any patients who have no diagnosis

In [7]:
dataset06_1=dataset_feb13[["patient_id","diagnosis_coded"]]
df_null=dataset06_1[dataset06_1['diagnosis_coded'].isnull()]
print(len(df_null["patient_id"].unique()),"number of patients have no diagnosis")

0 number of patients have no diagnosis


## Finding number of patients who have multiple diagnosis in this dataset

In [8]:
list_unique_patients=list(dataset_feb13["patient_id"].unique())
list_all_patients=list(dataset_feb13["patient_id"].values)
i=0
while len(list_unique_patients) > 0:
    patient_unique_number=len(list_unique_patients)
    print(patient_unique_number,"patients have at least",i+1,"diagnosis")
    uniquep=list_unique_patients
    allp=list_all_patients
    for k in range(len(uniquep)):
        #print(k)
        for j in range(len(allp)):
            if uniquep[k]==allp[j]:
                allp.remove(allp[j])
                break
    list_unique_patients=list(set(allp))
    list_all_patients=allp
    i=i+1        

10000 patients have at least 1 diagnosis
2298 patients have at least 2 diagnosis
567 patients have at least 3 diagnosis
120 patients have at least 4 diagnosis
27 patients have at least 5 diagnosis
6 patients have at least 6 diagnosis
3 patients have at least 7 diagnosis


# Creating columns for each concepts which have values in value_numeric column

### Preprocessing the dataset

In [9]:
dataset_feb13_1=dataset_feb04.drop(['value_coded'], axis=1) #we do not need value_coded column anymore
dataset_feb13_1["value_numeric"] = dataset_feb13_1["value_numeric"].str.replace(r'\\N','np.nan', regex=True) #replacing \N with 0
dataset_feb13_1=dataset_feb13_1.replace("np.nan",np.nan)
dataset_feb13_1["value_numeric"]=dataset_feb13_1['value_numeric'].astype(float) #converting the datatypes of value_numeric as float
dataset_feb13_1=dataset_feb13_1.dropna()
dataset_feb13_1=dataset_feb13_1.drop_duplicates()
dataset_feb13_1=dataset_feb13_1.reset_index(drop=True)
dataset_feb13_1

,patient_id,diagnosis_coded,concept_id,value_numeric
0,16359994,1385083,1482681,0.883785
1,16359994,1385083,1520177,74.000000
2,16359994,1385083,1520177,75.000000
3,16359994,1385083,1520177,77.000000
4,16359994,1385083,1520177,69.000000
...,...,...,...,...
6330662,19999784,1384995,1520694,31.500000
6330663,19999784,1384995,1520694,31.900000
6330664,19999784,1384995,1520694,32.100000
6330665,19999784,1384995,1520696,160.000000


### Getting value_numeric in different columns using get_dummies

In [10]:
dataset_feb13_1["concept_id"] = dataset_feb13_1["concept_id"].astype("category")
dataset_feb13_1['concept_id'].dtypes
dummies_1 = pd.get_dummies(dataset_feb13_1['concept_id'])
dummies_1=dummies_1.replace(0,np.nan)
dummies_1=dummies_1.astype(float)
dummies_1.values[dummies_1 == 1.0] = dataset_feb13_1['value_numeric']
dummies_1

,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,1520184,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
0,0.883785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6330662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6330663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6330664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6330665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Taking the patient_id column only to merge with dummies dataframe

In [11]:
data_id_only=dataset_feb13_1[['patient_id']]
dataframe_numeric=pd.concat([data_id_only,dummies_1],axis=1)
dataframe_numeric=dataframe_numeric.dropna(axis=1,how='all')

### Now if there are multiple rows for one patient, we are squeezing the dataframe to reduce the total number of rows(and making it equal to the number of patients) by taking the mean value of the column values for individual patient

In [12]:
squeezed_numeric= dataframe_numeric.groupby('patient_id').agg('mean')
squeezed_numeric=squeezed_numeric.reset_index()
squeezed_numeric

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
0,16359994,0.883785,1.550000,78.500000,117.0,129.0,NaN,NaN,NaN,16.100000,...,79.40,174.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16360107,2.967860,2.200000,92.000000,38.4,NaN,2454.0,NaN,7.266667,15.500000,...,84.85,186.65,69.5,176.5,NaN,NaN,NaN,NaN,NaN,NaN
2,16360581,NaN,NaN,96.000000,NaN,NaN,NaN,NaN,NaN,11.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16360590,2.304110,NaN,84.500000,NaN,NaN,NaN,NaN,NaN,11.000000,...,79.30,174.50,65.0,165.0,NaN,NaN,NaN,NaN,NaN,NaN
4,16360714,NaN,NaN,90.000000,NaN,NaN,NaN,NaN,NaN,11.857143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19998350,NaN,2.100000,81.666667,NaN,32.0,NaN,NaN,NaN,13.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,19998497,NaN,1.500000,91.100000,24.0,199.5,NaN,NaN,NaN,15.600000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,19999379,NaN,NaN,92.000000,NaN,NaN,NaN,NaN,NaN,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,19999625,0.960741,0.100000,85.666667,65.0,NaN,NaN,NaN,NaN,11.833333,...,50.50,111.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Creating columns for each concepts which have values in value_coded column

### Preprocessing the dataset

In [13]:
dataset_feb13_2=dataset_feb04.drop(['value_numeric'], axis=1) #we do not need value_numeric column anymore
dataset_feb13_2["value_coded"] = dataset_feb13_2["value_coded"].str.replace(r'\\N','np.nan', regex=True) #replacing \N with 0
dataset_feb13_2=dataset_feb13_2.replace("np.nan",np.nan)
#dataset_feb13_2["value_coded"]=dataset_feb13_2['value_coded'].astype(float) #converting the datatypes of value_numeric as float
dataset_feb13_2=dataset_feb13_2.dropna()
dataset_feb13_2=dataset_feb13_2.drop_duplicates()
dataset_feb13_2=dataset_feb13_2.reset_index(drop=True)
dataset_feb13_2

,patient_id,diagnosis_coded,concept_id,value_coded
0,16359994,1385083,1520778,1502470
1,16360107,1385109,1520261,1502535
2,16360107,1385109,1520288,1502467
3,16360107,1385109,1520288,1502548
4,16360107,1385109,1520333,1502519
...,...,...,...,...
80737,19998497,1397715,1520417,1502499
80738,19999625,1385121,1520417,1502499
80739,19999625,1385121,1520778,1502470
80740,19999784,1384995,1520731,1502522


### Getting value_numeric in different columns using get_dummies

In [ ]:
dataset_feb13_2["concept_id"] = dataset_feb13_2["concept_id"].astype("category")
dataset_feb13_2['concept_id'].dtypes
dummies_2 = pd.get_dummies(dataset_feb13_2['concept_id'])
dummies_2=dummies_2.replace(0,np.nan)
dummies_2=dummies_2.astype(float)
dummies_2.values[dummies_2 == 1.0] = dataset_feb13_2['value_coded']
dummies_2=dummies_2.fillna(3178353473).astype(int)
#dummies_2 = dummies_2.astype('int')
dummies_2

In [15]:
concept_coded_names=pd.read_csv("concept_coded_name.csv")
value_coded=concept_coded_names["Code"].unique().tolist()
name_coded=concept_coded_names["Name"].unique().tolist()
#len(value_coded)
for i in range(len(value_coded)):
    dummies_2=dummies_2.replace(value_coded[i],name_coded[i])
dummies_2=dummies_2.replace(3178353473,np.nan)
dummies_2

,1520187,1520206,1520214,1520215,1520234,1520256,1520261,1520274,1520276,1520287,...,1520778,1520779,1520781,1520782,1520784,1520785,1520786,1520787,1520789,1520792
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Venipuncture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,Clear,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Venipuncture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Taking the patient_id column only to merge with dummies dataframe

In [16]:
data_id_only=dataset_feb13_2[['patient_id']]
dataframe_coded=pd.concat([data_id_only,dummies_2],axis=1)
dataframe_coded=dataframe_coded.dropna(axis=1,how='all')
dataframe_coded

,patient_id,1520187,1520206,1520214,1520215,1520234,1520256,1520261,1520274,1520276,...,1520778,1520779,1520781,1520782,1520784,1520785,1520786,1520787,1520789,1520792
0,16359994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Venipuncture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16360107,NaN,NaN,NaN,NaN,NaN,NaN,Clear,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16360107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16360107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16360107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80737,19998497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80738,19999625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80739,19999625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Venipuncture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80740,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Using OneHotEncoder to encode the categorical variables in categorical columns

In [17]:
new_df = dataframe_coded.drop(['patient_id'],axis=1)
#new_df

# checking features
cat = new_df.select_dtypes(include='O').keys()
# display variabels
print("Number of columns in the dataframe that have categorical columns are:",len(cat))

# unique values in each columns
total_unique_values=0
for x in new_df.columns:
    #prinfting unique values
    #print("Column",x ,':', "Number of unique values in this column",len(new_df[x].unique()))
    #print(new_df[x].unique())
    total_unique_values=total_unique_values+len(new_df[x].unique())
    
print(total_unique_values-len(new_df.columns), "new columns should be added to the dataframe to add the OneHotEncoded codes.")


skencoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
sk_data = skencoder.fit_transform(new_df)
sk_data, sk_data.shape

list_encoded_columns=skencoder.get_feature_names_out().tolist()
list_encoded_columns

list_nan_columns=[]
list_code_columns=[]
for i in range(len(list_encoded_columns)):
    if (list_encoded_columns[i][-3:])=='nan':
        #print("yes")
        list_nan_columns.append(1)
    else:
        #print("no")
        list_code_columns.append(1)
        
print("Number of NaN columns",len(list_nan_columns),"\n")
print("Number of Categorical Code columns",len(list_code_columns),"\n")
print("We need to keep only the categorical code columns.")


sk_data_df=pd.DataFrame(sk_data)

for i in range(len(list_encoded_columns)):
    if (list_encoded_columns[i][-3:])=='nan':
        sk_data_df=sk_data_df.drop([i],axis=1)

sk_data_df  

sk_data_df["patient_id"]=dataframe_coded["patient_id"].values

sk_data_df
#df=pd.merge(df,df1,on="patient_id")

Number of columns in the dataframe that have categorical columns are: 114
239 new columns should be added to the dataframe to add the OneHotEncoded codes.
Number of NaN columns 114 

Number of Categorical Code columns 239 

We need to keep only the categorical code columns.


,0,1,3,4,6,8,10,12,14,15,...,339,341,342,344,345,347,348,350,351,patient_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16359994
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16360107
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16360107
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16360107
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16360107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19998497
80738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19999625
80739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19999625
80740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19999784


### Now if there are multiple rows for one patient, we are squeezing the dataframe to reduce the total number of rows(and making it equal to the number of patients) by taking the max of the column values for individual patient (thus if there is 1/present, it will come up even if the patient got 0 for the same feature some other time)

In [18]:
squeezed_coded= sk_data_df.groupby('patient_id').agg('max')
squeezed_coded=squeezed_coded.reset_index()
squeezed_coded

,patient_id,0,1,3,4,6,8,10,12,14,...,337,339,341,342,344,345,347,348,350,351
0,16359994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16360107,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,16360590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16360714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16361265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6809,19997887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6810,19997922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6811,19998497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6812,19999625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Now merging these two dataframes

In [22]:
# final=pd.merge(squeezed_numeric, squeezed_coded, on="patient_id")
# final
data_numeric=squeezed_numeric.set_index('patient_id')
data_numeric
data_coded=squeezed_coded.set_index('patient_id')
#here 0.0 means NaN, thus replacing it with np.nan
data_coded=data_coded.replace({0.0:np.nan})
data_coded

,0,1,3,4,6,8,10,12,14,15,...,337,339,341,342,344,345,347,348,350,351
patient_id,,,,,,,,,,,,,,,,,,,,,
16359994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16360107,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN
16360590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16360714,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16361265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19997887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19997922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
19998497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
data_coded=data_coded.astype("object")
data_coded.dtypes

0      object
1      object
3      object
4      object
6      object
        ...  
345    object
347    object
348    object
350    object
351    object
Length: 239, dtype: object

In [24]:
#pd.merge(data_numeric, data_coded,left_index=True, right_index=True)
df2 = pd.concat([data_numeric, data_coded], axis=1)
df2.reset_index(inplace=True)
df2

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,337,339,341,342,344,345,347,348,350,351
0,16359994,0.883785,1.550000,78.500000,117.0,129.0,NaN,NaN,NaN,16.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16360107,2.967860,2.200000,92.000000,38.4,NaN,2454.0,NaN,7.266667,15.500000,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN
2,16360581,NaN,NaN,96.000000,NaN,NaN,NaN,NaN,NaN,11.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16360590,2.304110,NaN,84.500000,NaN,NaN,NaN,NaN,NaN,11.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16360714,NaN,NaN,90.000000,NaN,NaN,NaN,NaN,NaN,11.857143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19998350,NaN,2.100000,81.666667,NaN,32.0,NaN,NaN,NaN,13.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,19998497,NaN,1.500000,91.100000,24.0,199.5,NaN,NaN,NaN,15.600000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,19999379,NaN,NaN,92.000000,NaN,NaN,NaN,NaN,NaN,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,19999625,0.960741,0.100000,85.666667,65.0,NaN,NaN,NaN,NaN,11.833333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Making the target variable dataframe

In [25]:
dataset_feb13=dataset_feb13.astype(str)
dataset_feb13_dia=dataset_feb13
dataset_feb13_dia=dataset_feb13_dia.replace({'1385187':1,'1385109':2,'1384870':3,'1384841':4,'1384995':5,'1397715':6,'1384886':7,'1474651':8,'1385083':9,'1385121':10})
dataset_feb13_dia["diagnosis_coded"] = dataset_feb13_dia["diagnosis_coded"].astype("category")
dataset_feb13_dia['diagnosis_coded'].dtypes
dummies_3 = pd.get_dummies(dataset_feb13_dia['diagnosis_coded'])
#dummies_3=dummies_3.replace(0,np.nan)
dummies_3=dummies_3.astype(float)
dummies_3.values[dummies_3 == 1.0] = dataset_feb13_dia['diagnosis_coded']
# dummies_3=dummies_2.fillna(3178353473).astype(int)
dummies_3 = dummies_3.astype('int')
dummies_3

,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,9,0
1,0,2,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,10
3,0,0,0,0,0,0,0,8,0,0
4,0,0,0,0,0,0,7,0,0,0
...,...,...,...,...,...,...,...,...,...,...
13016,0,0,0,0,0,6,0,0,0,0
13017,0,0,0,0,0,6,0,0,0,0
13018,0,0,3,0,0,0,0,0,0,0
13019,0,0,0,0,0,0,0,0,0,10


In [26]:
data_id_only=dataset_feb13[['patient_id']]
dataframe_coded_3=pd.concat([data_id_only,dummies_3],axis=1)
dataframe_coded_3=dataframe_coded_3.dropna(axis=1,how='all')
dataframe_coded_3

,patient_id,1,2,3,4,5,6,7,8,9,10
0,16359994,0,0,0,0,0,0,0,0,9,0
1,16360107,0,2,0,0,0,0,0,0,0,0
2,16360107,0,0,0,0,0,0,0,0,0,10
3,16360107,0,0,0,0,0,0,0,8,0,0
4,16360581,0,0,0,0,0,0,7,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
13016,19998497,0,0,0,0,0,6,0,0,0,0
13017,19999379,0,0,0,0,0,6,0,0,0,0
13018,19999379,0,0,3,0,0,0,0,0,0,0
13019,19999625,0,0,0,0,0,0,0,0,0,10


In [27]:
squeezed_diagnosis= dataframe_coded_3.groupby('patient_id').agg('max')
squeezed_diagnosis=squeezed_diagnosis.reset_index()
squeezed_diagnosis

,patient_id,1,2,3,4,5,6,7,8,9,10
0,16359994,0,0,0,0,0,0,0,0,9,0
1,16360107,0,2,0,0,0,0,0,8,0,10
2,16360581,0,0,0,0,0,0,7,0,0,0
3,16360590,0,0,3,0,0,0,0,0,0,0
4,16360714,0,0,0,0,0,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...
9995,19998350,0,2,0,0,0,0,0,0,0,0
9996,19998497,0,2,3,0,0,6,0,0,0,0
9997,19999379,0,0,3,0,0,6,0,0,0,0
9998,19999625,0,0,0,0,0,0,0,0,0,10


In [28]:
squeezed_diagnosis["patient_id"]=squeezed_diagnosis["patient_id"].astype(int)

In [29]:
merged=pd.merge(df2,squeezed_diagnosis,on="patient_id")
merged

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1_y,2,3_y,4_y,5,6_y,7,8_y,9,10_y
0,16359994,0.883785,1.550000,78.500000,117.0,129.0,NaN,NaN,NaN,16.100000,...,0,0,0,0,0,0,0,0,9,0
1,16360107,2.967860,2.200000,92.000000,38.4,NaN,2454.0,NaN,7.266667,15.500000,...,0,2,0,0,0,0,0,8,0,10
2,16360581,NaN,NaN,96.000000,NaN,NaN,NaN,NaN,NaN,11.500000,...,0,0,0,0,0,0,7,0,0,0
3,16360590,2.304110,NaN,84.500000,NaN,NaN,NaN,NaN,NaN,11.000000,...,0,0,3,0,0,0,0,0,0,0
4,16360714,NaN,NaN,90.000000,NaN,NaN,NaN,NaN,NaN,11.857143,...,0,0,0,0,0,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19998350,NaN,2.100000,81.666667,NaN,32.0,NaN,NaN,NaN,13.333333,...,0,2,0,0,0,0,0,0,0,0
9996,19998497,NaN,1.500000,91.100000,24.0,199.5,NaN,NaN,NaN,15.600000,...,0,2,3,0,0,6,0,0,0,0
9997,19999379,NaN,NaN,92.000000,NaN,NaN,NaN,NaN,NaN,13.000000,...,0,0,3,0,0,6,0,0,0,0
9998,19999625,0.960741,0.100000,85.666667,65.0,NaN,NaN,NaN,NaN,11.833333,...,0,0,0,0,0,0,0,0,0,10


In [524]:
merged.to_csv("merged_modified2.csv")